In [ ]:
# TEST 7: Visualisation
# ABSTRACT--------------------------------------------------------------------------------------------------------------------
This test visualizes solver outputs as ordered vehicle routes to verify continuity and physical validity.
# METHODOLOGY-----------------------------------------------------------------------------------------------------------------
Solver selected decision variables are parsed to extract vehicle movement. For each vehicle, edges 
we will reconstruct the full route in arrow form (e.g., 0 → i → j → 0).
# ADDED CODE------------------------------------------------------------------------------------------------------------------
from collections import defaultdict
sample = result.first.sample
# collect edges per vehicle
edges = defaultdict(list)

for var, val in sample.items():
    if val == 1 and var.startswith("x_"):
        _, i, j, k = var.split("_")
        edges[k].append((int(i), int(j)))

# build ordered routes
for k, e in edges.items():
    route = [0]  # start at depot
    while True:
        last = route[-1]
        next_nodes = [j for i, j in e if i == last]
        if not next_nodes:
            break
        route.append(next_nodes[0])
        if next_nodes[0] == 0:
            break

    print(f"Vehicle {k}: " + " → ".join(map(str, route)))
# RESULTS---------------------------------------------------------------------------------------------------------------------
Below I have left a result that passed the test. However, the prototype has failed this test. It has
shown several subtours.
# CONCLUSION------------------------------------------------------------------------------------------------------------------
Although the solver returned solutions that satisfied several individual constraints, the reconstructed
routes revealed that subroutes occur. A subtour elimination mechanism is necessary.

In [5]:
import dimod
from dimod import ConstrainedQuadraticModel, Binary
from dimod import quicksum

# Depot, Customers 1 and 2, Vehicles a and b
nodes = [0, 1, 2, 3, 4]          # 0 = depot
customers = [1, 2, 3, 4]
vehicles = ['a', 'b']

# Time dictionary (cost)
import itertools
import random
def GenerateDummyT(Nlocations, min_cost, max_cost, symmetric):
    t = {}
    for i, j in itertools.permutations(range(Nlocations), 2):
        if symmetric and (j, i) in t:
            t[(i, j)] = t[(j, i)]
        else:
            t[(i, j)] = random.randint(min_cost, max_cost)

    return t
t = GenerateDummyT(7, 1, 5, True)


# Create binary decision variables                   
# ---------------------------------
x = {}
for i in nodes:
    for j in nodes:
        if i != j: 
            for k in vehicles:
                name = f"x_{i}_{j}_{k}"
                x[i,j,k] = Binary(name)


# Building the CQM
# -----------------
cqm = ConstrainedQuadraticModel()

# 1) Objective: minimize total travel time
objective = quicksum(t[i,j] * x[i,j,k]
                     for i,j,k in x.keys())
cqm.set_objective(objective)

# Constraints
# -------------

# Each customer visited once
for v in customers:
    cqm.add_constraint(
        quicksum(x[i,v,k] for i in nodes if i != v for k in vehicles) == 1,
        label=f"Visit_{v}"
    )

# Start and end at Depot for each vehicle
for k in vehicles:
    # start from depot once
    cqm.add_constraint(
        quicksum(x[0,j,k] for j in customers) == 1,
        label=f"Start_{k}"
    )
    # return to depot once
    cqm.add_constraint(
        quicksum(x[i,0,k] for i in customers) == 1,
        label=f"End_{k}"
    )

# Flow conservation
for v in customers:
    for k in vehicles:
        cqm.add_constraint(
            quicksum(x[i,v,k] for i in nodes if i != v) -
            quicksum(x[v,j,k] for j in nodes if j != v) == 0,
            label=f"flow_{v}_{k}"
        )


# Converting to BQM 
bqm, invert = dimod.cqm_to_bqm(cqm)
solver = dimod.SimulatedAnnealingSampler()
result = solver.sample(bqm)

#---TEST----------------------------------------------------------------------
from collections import defaultdict

sample = result.first.sample

# collect edges per vehicle
edges = defaultdict(list)

for var, val in sample.items():
    if val == 1 and var.startswith("x_"):
        _, i, j, k = var.split("_")
        edges[k].append((int(i), int(j)))

# build ordered routes
for k, e in edges.items():
    route = [0]  # start at depot
    while True:
        last = route[-1]
        next_nodes = [j for i, j in e if i == last]
        if not next_nodes:
            break
        route.append(next_nodes[0])
        if next_nodes[0] == 0:
            break

    print(f"Vehicle {k}: " + " → ".join(map(str, route)))



Vehicle a: 0 → 3 → 0
Vehicle b: 0 → 4 → 1 → 2 → 0
